In [1]:
import os
import json
import base64

from openai import OpenAI
from dotenv import load_dotenv
from pydantic import BaseModel
from prompts import *

load_dotenv()

True

In [2]:
client = OpenAI(
    # base_url="https://api.together.xyz/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

In [3]:
def encode_image(image_path: str) -> str:
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

In [4]:
class Block(BaseModel):
    block_name: str
    description: str
    components: list[str]
    number_in_diagram: int | None
    position_in_diagram: str

class BlockWrapper(BaseModel):
    blocks: list[Block]

In [5]:
res = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": block_system_prompt},
        {"role": "user", "content": [
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image('../arch1.png')}"
                }
            }
        ]}
    ],
    response_format=BlockWrapper,
)


In [6]:
json.loads(res.choices[0].message.content)["blocks"]

[{'block_name': 'Amazon Location SDK',
  'description': 'SDK integrated with web app for location services.',
  'components': ['web app', 'Amazon Location SDK'],
  'number_in_diagram': None,
  'position_in_diagram': 'top-left'},
 {'block_name': 'AWS Amplify',
  'description': 'Used to build and deploy web applications.',
  'components': ['AWS Amplify'],
  'number_in_diagram': 1,
  'position_in_diagram': 'top-center'},
 {'block_name': 'AWS AppSync',
  'description': 'Manages and synchronizes data between users via API.',
  'components': ['AWS AppSync'],
  'number_in_diagram': 2,
  'position_in_diagram': 'center'},
 {'block_name': 'Amazon DynamoDB',
  'description': 'Stores and retrieves application data.',
  'components': ['Amazon DynamoDB'],
  'number_in_diagram': 3,
  'position_in_diagram': 'right-center'},
 {'block_name': 'Amazon Location Service',
  'description': 'Provides map and tracking services.',
  'components': ['Track', 'Map', 'Geofence', 'Amazon Location Service'],
  'numbe

In [7]:
blocks = res.choices[0].message.parsed.blocks

In [8]:
print(blocks[0].model_dump_json(indent=4))

{
    "block_name": "Amazon Location SDK",
    "description": "SDK integrated with web app for location services.",
    "components": [
        "web app",
        "Amazon Location SDK"
    ],
    "number_in_diagram": null,
    "position_in_diagram": "top-left"
}


In [9]:
class ConnInfo(BaseModel):
    block_name: str | None
    component_name: str | None

class Connection(BaseModel):
    source: ConnInfo | None
    target: ConnInfo | None
    direction: str | None
    description: str | None

class ConnectionWrapper(BaseModel):
    connections: list[Connection]

In [10]:
print(blocks[0].model_dump_json(indent=4))


{
    "block_name": "Amazon Location SDK",
    "description": "SDK integrated with web app for location services.",
    "components": [
        "web app",
        "Amazon Location SDK"
    ],
    "number_in_diagram": null,
    "position_in_diagram": "top-left"
}


In [11]:
blocks_str = "\n".join(b.model_dump_json(indent=4) for b in blocks)

In [12]:
conns = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": connection_system_prompt.format(blocks_info=blocks_str)},
        {
            "role": "user", "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image('../arch1.png')}"
                    }
                }
            ]
        }
    ],
    response_format=ConnectionWrapper,
)

In [13]:
print(conns.choices[0].message.parsed.connections[0].model_dump_json(indent=4))


{
    "source": {
        "block_name": "Amazon Location SDK",
        "component_name": "web app"
    },
    "target": {
        "block_name": "AWS Amplify",
        "component_name": "AWS Amplify"
    },
    "direction": "one-way",
    "description": "The web app uses AWS Amplify for deployment and management."
}


In [14]:
conns.choices[0].message.parsed.connections

[Connection(source=ConnInfo(block_name='Amazon Location SDK', component_name='web app'), target=ConnInfo(block_name='AWS Amplify', component_name='AWS Amplify'), direction='one-way', description='The web app uses AWS Amplify for deployment and management.'),
 Connection(source=ConnInfo(block_name='Amazon Location SDK', component_name='Amazon Location SDK'), target=ConnInfo(block_name='web app', component_name='web app'), direction='bidirectional', description='Integrates location services into the web app.'),
 Connection(source=ConnInfo(block_name='AWS Amplify', component_name='AWS Amplify'), target=ConnInfo(block_name='AWS AppSync', component_name='AWS AppSync'), direction='one-way', description='Amplify connects to AppSync for synchronizing data.'),
 Connection(source=ConnInfo(block_name='AWS AppSync', component_name='AWS AppSync'), target=ConnInfo(block_name='Amazon DynamoDB', component_name='Amazon DynamoDB'), direction='one-way', description='AppSync manages data stored in DynamoD

In [16]:
master_prompt = ""

In [17]:
for block in blocks:
    master_prompt += block.model_dump_json(indent=4)
    master_prompt += "\n\n"
    for conn in conns.choices[0].message.parsed.connections:
        if conn.source.block_name == block.block_name:
            master_prompt += conn.model_dump_json(indent=4)
            master_prompt += "\n\n"

In [18]:
print(master_prompt)

{
    "block_name": "Amazon Location SDK",
    "description": "SDK integrated with web app for location services.",
    "components": [
        "web app",
        "Amazon Location SDK"
    ],
    "number_in_diagram": null,
    "position_in_diagram": "top-left"
}

{
    "source": {
        "block_name": "Amazon Location SDK",
        "component_name": "web app"
    },
    "target": {
        "block_name": "AWS Amplify",
        "component_name": "AWS Amplify"
    },
    "direction": "one-way",
    "description": "The web app uses AWS Amplify for deployment and management."
}

{
    "source": {
        "block_name": "Amazon Location SDK",
        "component_name": "Amazon Location SDK"
    },
    "target": {
        "block_name": "web app",
        "component_name": "web app"
    },
    "direction": "bidirectional",
    "description": "Integrates location services into the web app."
}

{
    "block_name": "AWS Amplify",
    "description": "Used to build and deploy web applications.",
   